In [12]:
from torchvision import models                      
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import torchvision.transforms as tf
import segmentation_models_pytorch
from PIL import Image
from os import listdir
from os.path import isfile, join
from pathlib import Path
import cv2
import matplotlib.pyplot as plt

device = torch.device('cuda') if torch.cuda.is_available() else torch.device(
    'cpu')  # Check if there is GPU if not set trainning to CPU (very slow)


modelPath = "4000.torch"
width, height = 256, 256  # image width and height

Learning_Rate=1e-5
batchSize=16

transformImg = tf.Compose([tf.Resize((height, width)), tf.ToTensor(), tf.Normalize((0.485, 0.456, 0.406),
                                                                                   (0.229, 0.224, 0.225))])


data_paths_images = Path("/media/artem/A2F4DEB0F4DE85C7/myData/datasets/barcodes_hw/Test-20220427T080744Z-001/Test/Images")

onlyfiles_paths_images = {f: join(data_paths_images, f) for f in listdir(data_paths_images) if isfile(join(data_paths_images, f))}

seg_model = segmentation_models_pytorch.UnetPlusPlus(in_channels=3, classes=1)
seg_model.load_state_dict(torch.load(modelPath))
seg_model.to(device).eval()

def getSubImage(best_area, src):
    # Get center, size, and angle from rect
    pts1 = np.float32(best_area)
    pts2 = np.float32([[300, 300], [0, 300], [0, 0], [300, 0]])

    M = cv2.getPerspectiveTransform(pts1, pts2)
    
    out = cv2.warpPerspective(src, M, (300, 300))

    img90 = np.rot90(out)
    img180 = np.rot90(img90)
    img_n270 = np.rot90(img180)
    img = out

    return img, img90, img180, img_n270

def cutBarcode(x):
    width_orgin, height_orgin = x.size[0], x.size[1]  # Get image original size

    Img = transformImg(x)  # Transform to pytorch
    Img = torch.autograd.Variable(Img, requires_grad=False).to(device).unsqueeze(0)

    with torch.no_grad():
        Prd = seg_model(Img) 

    Prd = tf.Resize((height_orgin, width_orgin))(Prd[0])[0]  # Resize to origninal size

    mask = Prd.detach().cpu().numpy() > 0.5
    mask_img = mask.astype(np.uint8)

    w, h = mask_img.shape

    mask_img *= 255
    
    Img_orig = x
    contours, _ = cv2.findContours(mask_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    best_area = []
    max_S = -1
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > max_S:
            epsilon = 0.1*cv2.arcLength(cnt,True)

            
            tmp_area = cv2.approxPolyDP(cnt,epsilon,True)
            if len(tmp_area) == 4:
                best_area = tmp_area
                max_S = area
   
    if len(best_area) != 4:
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > max_S:
                epsilon = 0.1*cv2.arcLength(cnt,True)

                tmp_area = cv2.approxPolyDP(cnt,epsilon,True)
                best_area = []
                
                best_area.append(tmp_area[0])
                best_area.append([[tmp_area[0][0][0], tmp_area[1][0][1]]])
                best_area.append(tmp_area[1])
                best_area.append([[tmp_area[1][0][0], tmp_area[0][0][1]]])
                max_S = area
    
    #Img_orig = np.array(Img_orig)
    #barcodes = getSubImage(best_area, Img_orig)[0]
    
    #plt.imshow(barcodes)  # display image
    #plt.show()
    
    return best_area

def getCords(onlyfiles_paths_images, name):  # First lets load random image and  the corresponding annotation    
    img_file_path =  onlyfiles_paths_images[name]
    
    crop_cords =  cutBarcode(Image.open(img_file_path, mode='r', formats=None))
    return crop_cords



In [32]:
import pandas as pd

dataFrame = pd.read_csv("/home/artem/PycharmProjects/course_ocr/course_ocr/task3/ref_answer.csv", encoding="utf-16")

ans = []

for idx, row in dataFrame.iterrows(): # Training loop
    name = row[0]
    row = []
    print(name)
    cords = getCords(onlyfiles_paths_images, name)
    row.append(name)
    row.append("0000000000000")
    
    row.append(str(int(cords[0][0][0])))
    row.append(str(int(cords[0][0][1])))
    
    
    row.append(str(int(cords[1][0][0])))
    row.append(str(int(cords[1][0][1])))
    
    row.append(str(int(cords[2][0][0])))
    row.append(str(int(cords[2][0][1])))
    
    row.append(str(int(cords[3][0][0])))
    row.append(str(int(cords[3][0][1])))
    
    row.append("1111111111111111111111111111111111111111111111111")
    ans.append(row)

ans_csv = pd.DataFrame(ans)

ans_csv.to_csv("/home/artem/PycharmProjects/course_ocr/course_ocr/task3/answer.csv", header=False, encoding="utf-16", index=False)

f9bf5d96a0cf5d64aac4eb5806554224.png
a154b624c06072da1a01c97ebc3894c1.png
749003091876fc58be69db54bbaae8f1.png
560ca4ac1260313eb5979140c4f9ef31.png
2bc877416a54aea8a94227ce9d1e5d1f.png
589b454c920322816d8df0494cb07bba.png
cd82f250fa1ff45ae2eada5d91ed90a2.png
2ebb873b5b1a0c782e8432cf3a82d71f.png
36824964d90c9e9ebc65eeb1b0438c66.png
365df19a3155c1019ea182329a074f47.png
c3c5e1e6da39334c32873c0cf4eeb9ec.png
54c516c07afe31075266f04a04d2d03a.png
7ba840fa58126ec78764d77ee9720a27.png
fedcc45b69bb6c40c4599bbb499ee365.png
6c0923f882043d8ab709537da2813042.png
aab7bd3438de5ce40c4c7de3db0ecf23.png
7711af7efa618bf29db3e2ad72ee534a.png
d0581f8681b200dc78ef271460905612.png
40ed826ebe0916f70999c831fc078faf.png
1cd72c7dcf77e1b8a401c0a7b41573c8.png
632cf29db1b4cfb252d20edba6508639.png
eddb324b906534e158177f022a1ca3b4.png
f7c811258e32c7150ff454bf419e8bc7.png
2de6d574d39d2c8fcb700ab62fa25e94.png
099574d0f797fe2519686fd1e724ebf3.png
429394b77b7a164916bbf2a8a824f30e.png
a6522bc3e9d7207451c564c3bb4a89b8.png
8